# ATE and ABSA for the all three drugs types
Code

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
cd ..


/


In [ ]:

cd drive/MyDrive/DrugReviews_ATE

/drive/MyDrive/DrugReviews_ATE


In [ ]:
import pandas as pd
a_file = 'arthritis_final.csv'#'depression_final.csv'
df1 = pd.read_csv(a_file)

df1 = df1[['Extracted Sentence', 'Encoded Terms', 'Polarities']]
df1 = df1.rename (columns = {"Extracted Sentence": "Tokens", "Encoded Terms":"Tags",'Polarities':'Polarities'})

df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1372 entries, 0 to 1371
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Tokens      1372 non-null   object
 1   Tags        1372 non-null   object
 2   Polarities  1372 non-null   object
dtypes: object(3)
memory usage: 32.3+ KB


In [ ]:
df2 = df1.head(1000)
df3 = df1.tail(372)

In [ ]:
df2.reset_index(drop=True, inplace=True)
df2.to_csv('/drive/MyDrive/DrugReviews_ATE/depression_train.csv', index=False)
df3.reset_index(drop=True, inplace=True)
df3.to_csv('/drive/MyDrive/DrugReviews_ATE/depression_test.csv', index=False)

In [ ]:
df3

,Tokens,Tags,Polarities
0,"['when', 'i', 'was', 'diagnosed', 'i', 'was', ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1..."
1,"['i', 'dont', 'remember', 'my', 'initial', 'do...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,"['i', 'have', 'been', 'using', 'daypro', 'for'...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 0]"
3,"['i', 'have', 'arthritis', 'in', 'my', 'wrists...","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[-1, -1, 2, -1, -1, 2, -1, -1, -1, -1, -1, -1,..."
4,"['not', 'better', 'than', 'bextra', 'but', 'le...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1]"
...,...,...,...
367,"['i', 'did', 'not', 'find', 'the', 'shot', 'pa...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
368,"['no', 'other', 'side', 'affects', 'thus', 'far']","[0, 0, 1, 2, 0, 0]","[-1, -1, 2, 2, -1, -1]"
369,"['i', 'tried', 'oral', 'therapy', 'methotrexat...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0..."
370,"['with', 'enbrel', 'the', 'stabbing', 'sensati...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.7 MB/s eta 0:00:00


In [ ]:
from torch.utils.data import DataLoader, ConcatDataset
from transformers import BertTokenizer
import torch
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import time
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix



---

#Now Coding for BERT_ATE

In [ ]:
from transformers import BertConfig, BertModel
import torch
class bert_ATE(torch.nn.Module):
    def __init__(self, pretrain_model):
        super(bert_ATE, self).__init__()
        self.bert = BertModel.from_pretrained(pretrain_model)
        self.linear = torch.nn.Linear(self.bert.config.hidden_size, 3)
        self.loss_fn = torch.nn.CrossEntropyLoss()

    def forward(self, ids_tensors, tags_tensors, masks_tensors):
        bert_outputs,_ = self.bert(input_ids=ids_tensors, attention_mask=masks_tensors,return_dict=False)
        #print(bert_outputs.size())
        #print(type(bert_outputs))
        #print(bert_outputs)
        linear_outputs = self.linear(bert_outputs)
        #print(linear_outputs.size())


        if tags_tensors is not None:
            tags_tensors = tags_tensors.view(-1)
            linear_outputs = linear_outputs.view(-1,3)
            print(linear_outputs.size())
            print(tags_tensors.size())
            loss = self.loss_fn(linear_outputs, tags_tensors)
            return loss
        else:
            return linear_outputs


class bert_ABSA(torch.nn.Module):
    def __init__(self, pretrain_model):
        super(bert_ABSA, self).__init__()
        self.bert = BertModel.from_pretrained(pretrain_model)
        self.linear = torch.nn.Linear(self.bert.config.hidden_size, 3)
        self.loss_fn = torch.nn.CrossEntropyLoss()

    def forward(self, ids_tensors, lable_tensors, masks_tensors, segments_tensors):
        _, pooled_outputs = self.bert(input_ids=ids_tensors, attention_mask=masks_tensors, token_type_ids=segments_tensors,return_dict=False)
        # print(bert_outputs.size())
        linear_outputs = self.linear(pooled_outputs)
        # print(linear_outputs.size())

        if lable_tensors is not None:
            # print(linear_outputs.size())
            # print(tags_tensors.size())
            loss = self.loss_fn(linear_outputs, lable_tensors)
            return loss
        else:
            return linear_outputs

In [ ]:
# Dataset ATM
from torch.utils.data import Dataset
import pandas as pd
import torch
from torch.utils.data import Dataset
import pandas as pd
import torch

class dataset_ATM(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        tokens, tags, pols = self.df.iloc[idx, :3].values



        tokens = tokens.replace("'", "").strip("][").split(', ')
        #print('token',tokens)
        tags = tags.strip('][').split(', ')
        pols = pols.strip('][').split(', ')


        bert_tokens = []
        bert_tags = []
        bert_pols = []
        for i in range(len(tokens)):
            t = self.tokenizer.tokenize(tokens[i])
            bert_tokens += t
            bert_tags += [int(tags[i])]*len(t)
            bert_pols += [int(pols[i])]*len(t)

        bert_ids = self.tokenizer.convert_tokens_to_ids(bert_tokens)

        ids_tensor = torch.tensor(bert_ids)
        tags_tensor = torch.tensor(bert_tags)
        pols_tensor = torch.tensor(bert_pols)

        return bert_tokens, ids_tensor, tags_tensor, pols_tensor

    def __len__(self):
        return len(self.df)


class dataset_ABSA(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        tokens, tags, pols = self.df.iloc[idx, :3].values
        #print('tokens:', tokens)
        #print('tags:', tags)
        #print('pols:', pols)
        bert_tokens = []
        bert_att = []
        pols_label = 0
        pols_tensor = torch.tensor([pols_label])
        tokens = tokens.replace("'", "").strip("][").split(',')
        tags = tags.strip('][').split(',')
        pols = pols.strip('][').split(',')

        #bert_tokens = []
        #bert_att = []
        #pols_label = 0
        for i in range(len(tokens)):
            t = self.tokenizer.tokenize(tokens[i])
            bert_tokens += t
            if int(pols[i]) != -1:
                bert_att += t
                pols_label = int(pols[i])

        segment_tensor = [0] + [0]*len(bert_tokens) + [0] + [1]*len(bert_att)
        bert_tokens = ['[cls]'] + bert_tokens + ['[sep]'] + bert_att


        bert_ids = self.tokenizer.convert_tokens_to_ids(bert_tokens)

        ids_tensor = torch.tensor(bert_ids)
        pols_tensor = torch.tensor(pols_label)
        segment_tensor = torch.tensor(segment_tensor)

        return bert_tokens, ids_tensor, segment_tensor, pols_tensor

    def __len__(self):
        return len(self.df)

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pretrain_model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(pretrain_model_name)
lr = 2e-5
model_ATE = bert_ATE(pretrain_model_name).to(DEVICE)
optimizer_ATE = torch.optim.Adam(model_ATE.parameters(), lr=lr)
model_ABSA = bert_ABSA(pretrain_model_name).to(DEVICE)
optimizer_ABSA = torch.optim.Adam(model_ABSA.parameters(), lr=lr)

In [ ]:
def evl_time(t):
    min, sec= divmod(t, 60)
    hr, min = divmod(min, 60)
    return int(hr), int(min), int(sec)

def load_model(model, path):
    model.load_state_dict(torch.load(path), strict=False)
    return model

def save_model(model, name):
    torch.save(model.state_dict(), name)

In [ ]:
train_ds = dataset_ATM(pd.read_csv("/drive/MyDrive/DrugReviews_ATE/depression_train.csv"), tokenizer)
test_ds = dataset_ATM(pd.read_csv("/drive/MyDrive/DrugReviews_ATE/depression_test.csv"), tokenizer)

In [ ]:
test_ds[0]

(['[cls]',
  'when',
  'i',
  'was',
  'diagnosed',
  'i',
  'was',
  'in',
  'severe',
  'pain',
  'unable',
  'to',
  'hold',
  'my',
  'infant',
  'child',
  'or',
  'drive',
  '[sep]',
  'pain'],
 tensor([  100,  2043,  1045,  2001, 11441,  1045,  2001,  1999,  5729,  3255,
          4039,  2000,  2907,  2026, 10527,  2775,  2030,  3298,   100,  3255]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
 tensor(0))

In [ ]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    tags_tensors = [s[2] for s in samples]
    tags_tensors = pad_sequence(tags_tensors, batch_first=True)

    pols_tensors = [s[3] for s in samples]
    pols_tensors = pad_sequence(pols_tensors, batch_first=True)

    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)

    return ids_tensors, tags_tensors, pols_tensors, masks_tensors

In [ ]:
train_loader = DataLoader(train_ds, batch_size=5, collate_fn=create_mini_batch, shuffle = True)
test_loader = DataLoader(test_ds, batch_size=50, collate_fn=create_mini_batch, shuffle = True)

In [ ]:
"""for batch in train_loader:
    w,x,y,z = batch
    print(w)
    print(w.size())
    print(x)
    print(x.size())
    print(y)
    print(y.size())
    print(z)
    print(z.size())
    break"""

'for batch in train_loader:\n    w,x,y,z = batch\n    print(w)\n    print(w.size())\n    print(x)\n    print(x.size())\n    print(y)\n    print(y.size())\n    print(z)\n    print(z.size())\n    break'

In [ ]:
def train_model_ATE(loader, epochs):
    all_data = len(loader)
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        correct_predictions = 0

        for data in loader:
            t0 = time.time()
            ids_tensors, tags_tensors, _, masks_tensors = data
            ids_tensors = ids_tensors.to(DEVICE)
            tags_tensors = tags_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)
            #print(type(ids_tensors)); print(type(tags_tensors)); print(type(masks_tensors))

            loss = model_ATE(ids_tensors=ids_tensors, tags_tensors=tags_tensors, masks_tensors=masks_tensors)
            losses.append(loss.item())
            loss.backward()
            optimizer_ATE.step()
            optimizer_ATE.zero_grad()

            finish_data += 1
            current_times.append(round(time.time()-t0,3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
            print('epoch:', epoch, " batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)

        save_model(model_ATE, 'bert_ATE.pkl')

def test_model_ATE(loader):
    pred = []
    trueth = []
    with torch.no_grad():
        for data in loader:

            ids_tensors, tags_tensors, _, masks_tensors = data
            ids_tensors = ids_tensors.to(DEVICE)
            tags_tensors = tags_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ATE(ids_tensors=ids_tensors, tags_tensors=None, masks_tensors=masks_tensors)

            _, predictions = torch.max(outputs, dim=2)

            pred += list([int(j) for i in predictions for j in i ])
            trueth += list([int(j) for i in tags_tensors for j in i ])

    return trueth, pred

In [ ]:
%time train_model_ATE(train_loader, 3)

torch.Size([140, 3])
torch.Size([140])
epoch: 0  batch: 1 / 200  loss: 1.1436004638671875  hr: 0  min: 40  sec: 28
torch.Size([130, 3])
torch.Size([130])
epoch: 0  batch: 2 / 200  loss: 1.0031709372997284  hr: 0  min: 30  sec: 50
torch.Size([185, 3])
torch.Size([185])
epoch: 0  batch: 3 / 200  loss: 0.8650495807329813  hr: 0  min: 29  sec: 14
torch.Size([140, 3])
torch.Size([140])
epoch: 0  batch: 4 / 200  loss: 0.7759258598089218  hr: 0  min: 27  sec: 27
torch.Size([80, 3])
torch.Size([80])
epoch: 0  batch: 5 / 200  loss: 0.6856137752532959  hr: 0  min: 26  sec: 45
torch.Size([195, 3])
torch.Size([195])
epoch: 0  batch: 6 / 200  loss: 0.6011197790503502  hr: 0  min: 28  sec: 55
torch.Size([135, 3])
torch.Size([135])
epoch: 0  batch: 7 / 200  loss: 0.5457715924297061  hr: 0  min: 29  sec: 8
torch.Size([160, 3])
torch.Size([160])
epoch: 0  batch: 8 / 200  loss: 0.500085974112153  hr: 0  min: 29  sec: 55
torch.Size([140, 3])
torch.Size([140])
epoch: 0  batch: 9 / 200  loss: 0.47277349730

In [ ]:
model_ATE = load_model(model_ATE, 'bert_ATE.pkl')

In [ ]:
%time x, y = test_model_ATE(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))


In [ ]:
confusion_matrix(x, y)

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

#labels=["0", "aspect_term1", "aspect_term2"]
cm = confusion_matrix(x, y)
print(cm)



---



---



#ABSA

In [ ]:
""""laptops_train_ds = dataset_ABSA(pd.read_csv("data/laptops_train.csv"), tokenizer)
laptops_test_ds = dataset_ABSA(pd.read_csv("data/laptops_test.csv"), tokenizer)
restaurants_train_ds = dataset_ABSA(pd.read_csv("data/restaurants_train.csv"), tokenizer)
restaurants_test_ds = dataset_ABSA(pd.read_csv("data/restaurants_test.csv"), tokenizer)
twitter_train_ds = dataset_ABSA(pd.read_csv("data/twitter_train.csv"), tokenizer)
twitter_test_ds = dataset_ABSA(pd.read_csv("data/twitter_test.csv"), tokenizer)""""

In [ ]:
train_ds = dataset_ABSA(pd.read_csv("/drive/MyDrive/DrugReviews_ATE/depression_train.csv"), tokenizer)
test_ds = dataset_ABSA(pd.read_csv("/drive/MyDrive/DrugReviews_ATE/depression_test.csv"), tokenizer)

In [ ]:
w,x,y,z = train_ds.__getitem__(121)
print(w)
print(len(w))
print(x)
print(len(x))
print(y)
print(len(y))
print(z)

['[cls]', 'it', 'works', 'extremely', 'well', 'for', 'me', 'one', 'of', 'my', 'side', 'effects', 'is', 'i', 'get', 'a', 'burst', 'of', 'energy', 'about', 'an', 'hour', 'after', 'i', 'take', 'it', '[sep]', 'effects']
28
tensor([ 100, 2009, 2573, 5186, 2092, 2005, 2033, 2028, 1997, 2026, 2217, 3896,
        2003, 1045, 2131, 1037, 6532, 1997, 2943, 2055, 2019, 3178, 2044, 1045,
        2202, 2009,  100, 3896])
28
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1])
28
tensor(0)


In [ ]:
def create_mini_batch2(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    segments_tensors = [s[2] for s in samples]
    segments_tensors = pad_sequence(segments_tensors, batch_first=True)

    label_ids = torch.stack([s[3] for s in samples])

    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)

    return ids_tensors, segments_tensors, masks_tensors, label_ids

In [ ]:
"""train_ds = ConcatDataset([laptops_train_ds, restaurants_train_ds, twitter_train_ds])
test_ds = ConcatDataset([laptops_test_ds, restaurants_test_ds, twitter_test_ds])
"""
train_loader = DataLoader(train_ds, batch_size=4, collate_fn=create_mini_batch2, shuffle = True)
test_loader = DataLoader(test_ds, batch_size=50, collate_fn=create_mini_batch2, shuffle = True)

In [ ]:
for batch in train_loader:
  w,x,y,z = batch
  print(w)
  print(w.size())
  print(x)
  print(x.size())
  print(y)
  print(y.size())
  print(z)
  print(z.size())
  break


tensor([[  100,  2045,  2001,  1037,  3809,  7312,  1999,  3255,  2007,  1996,
          2969, 13341,   100,  3255,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  100,  2065,  1999,  5729,  3255,  1045,  2097,  2036,  2202,  1037,
          6740,  9483,  2121,  2170,  4496, 21031,  2595,  1998,  1045,  2572,
          2583,  2000,  2717,  1998,  3853,  1998,  2131,  2172,  2734,  4335,
           100,  3255],
        [  100, 26261, 22943, 10854,  3768,  1997,  3637, 13862,  2229,  2021,
          2053,  5665,  3896,  2090,  5352,  2706,  4237,   100,  3896,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  100,  1045,  5060,  4308, 17964,  2130,  2295,  1045,  2202,  2009,
          2007,  1037,  7954,   100,  4308, 17964,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,  

In [ ]:
def train_model_ABSA(loader, epochs):
    all_data = len(loader)
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        correct_predictions = 0

        for data in loader:
            t0 = time.time()
            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            label_ids = label_ids.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)
            #label_ids = label_ids.squeeze(dim=-1).long()
            #loss = model_ABSA(ids_tensors=ids_tensors, masks_tensors=masks_tensors, segments_tensors=segments_tensors, lable_tensors=label_ids)
            #label_ids = label_ids.squeeze().long()
            #loss = model_ABSA(ids_tensors=ids_tensors, label_tensors=label_ids, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            loss = model_ABSA(ids_tensors=ids_tensors, lable_tensors=label_ids, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            losses.append(loss.item())
            loss.backward()
            optimizer_ABSA.step()
            optimizer_ABSA.zero_grad()

            finish_data += 1
            current_times.append(round(time.time()-t0,3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
            print('epoch:', epoch, " batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)

        save_model(model_ABSA, 'bert_ABSA2.pkl')

def test_model_ABSA(loader):
    pred = []
    trueth = []
    with torch.no_grad():
        for data in loader:

            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ABSA(ids_tensors, None, masks_tensors=masks_tensors, segments_tensors=segments_tensors)

            _, predictions = torch.max(outputs, dim=1)

            pred += list([int(i) for i in predictions])
            trueth += list([int(i) for i in label_ids])

    return trueth, pred

In [ ]:
%time train_model_ABSA(train_loader, 3)


epoch: 0  batch: 1 / 250  loss: 1.1939449310302734  hr: 1  min: 8  sec: 0
epoch: 0  batch: 2 / 250  loss: 1.2567203640937805  hr: 1  min: 2  sec: 9
epoch: 0  batch: 3 / 250  loss: 1.1587790648142497  hr: 0  min: 56  sec: 31
epoch: 0  batch: 4 / 250  loss: 1.1474730372428894  hr: 0  min: 47  sec: 42
epoch: 0  batch: 5 / 250  loss: 1.073958694934845  hr: 0  min: 42  sec: 56
epoch: 0  batch: 6 / 250  loss: 1.139422247807185  hr: 0  min: 41  sec: 20
epoch: 0  batch: 7 / 250  loss: 1.085720317704337  hr: 0  min: 38  sec: 55
epoch: 0  batch: 8 / 250  loss: 1.1611798256635666  hr: 0  min: 39  sec: 23
epoch: 0  batch: 9 / 250  loss: 1.2022702164120145  hr: 0  min: 40  sec: 33
epoch: 0  batch: 10 / 250  loss: 1.187657392024994  hr: 0  min: 40  sec: 18
epoch: 0  batch: 11 / 250  loss: 1.1781377792358398  hr: 0  min: 38  sec: 58
epoch: 0  batch: 12 / 250  loss: 1.1678131918112438  hr: 0  min: 37  sec: 30
epoch: 0  batch: 13 / 250  loss: 1.1405550745817332  hr: 0  min: 36  sec: 20
epoch: 0  batch:

# ATE + ABSA

In [ ]:
def predict_model_ABSA(sentence, aspect, tokenizer):
    t1 = tokenizer.tokenize(sentence)
    t2 = tokenizer.tokenize(aspect)

    word_pieces = ['[cls]']
    word_pieces += t1
    word_pieces += ['[sep]']
    word_pieces += t2

    segment_tensor = [0] + [0]*len(t1) + [0] + [1]*len(t2)

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)
    segment_tensor = torch.tensor(segment_tensor).to(DEVICE)

    with torch.no_grad():
        outputs = model_ABSA(input_tensor, None, None, segments_tensors=segment_tensor)
        _, predictions = torch.max(outputs, dim=1)

    return word_pieces, predictions, outputs

def predict_model_ATE(sentence, tokenizer):
    word_pieces = []
    tokens = tokenizer.tokenize(sentence)
    word_pieces += tokens

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)

    with torch.no_grad():
        outputs = model_ATE(input_tensor, None, None)
        _, predictions = torch.max(outputs, dim=2)
    predictions = predictions[0].tolist()

    return word_pieces, predictions, outputs

def ATE_ABSA(text):

    terms = []
    word = ""
    x, y, z = predict_model_ATE(text, tokenizer)
    for i in range(len(y)):
        if y[i] == 1:
            if len(word) != 0:
                terms.append(word.replace(" ##",""))
            word = x[i]
        if y[i] == 2:
            word += (" " + x[i])


    if len(word) != 0:
            terms.append(word.replace(" ##",""))

    print("tokens:", x)
    print("ATE:", terms)

    if len(terms) != 0:
        for i in terms:
            _, c, p = predict_model_ABSA(text, i, tokenizer)
            print("term:", [i], "class:", [int(c)], "ABSA:", [float(p[0][0]), float(p[0][1]), float(p[0][2])])


In [ ]:
%time x, y = test_model_ABSA(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))

In [ ]:
#checkpoint = torch.load(checkpoint, map_location=torch.device('cpu'))
model_ABSA = load_model(model_ABSA, 'bert_ABSA2.pkl')
model_ATE = load_model(model_ATE, 'bert_ATE.pkl')

In [ ]:
predict_model_ABSA

In [ ]:
text = "I had to stop taking it after a week because it upset my stomach causing vomiting, nausea, and extreme diarrhea."
ATE_ABSA(text)

tokens: ['i', 'had', 'to', 'stop', 'taking', 'it', 'after', 'a', 'week', 'because', 'it', 'upset', 'my', 'stomach', 'causing', 'vomiting', ',', 'nausea', ',', 'and', 'extreme', 'dia', '##rr', '##hea', '.']
ATE: ['stomach']
term: ['stomach'] class: [0] ABSA: [0.9560188055038452, -0.5063076019287109, 0.08479826152324677]


In [ ]:
text ="cleared me right up even with my throat hurting it went away after taking the medicine"
ATE_ABSA(text)

tokens: ['cleared', 'me', 'right', 'up', 'even', 'with', 'my', 'throat', 'hurting', 'it', 'went', 'away', 'after', 'taking', 'the', 'medicine']
ATE: ['throat']
term: ['throat'] class: [0] ABSA: [0.25615406036376953, -0.14244669675827026, 0.14468953013420105]


In [ ]:
text = "I have such heavy, long and painful periods. I have been suffering for a long time"
ATE_ABSA(text)

tokens: ['i', 'have', 'such', 'heavy', ',', 'long', 'and', 'painful', 'periods', '.', 'i', 'have', 'been', 'suffering', 'for', 'a', 'long', 'time']
ATE: ['periods']
term: ['periods'] class: [0] ABSA: [0.5847797989845276, 0.2184433937072754, -0.3567959666252136]
